In [12]:
import os
import struct
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# Set Plotly renderer
pio.renderers.default = "browser"

# File paths
file_paths = {
    "v1": "examples/v1/Dime.tmd",
    "v2": "examples/v2/Dime.tmd",
}

def check_version(file_path, num_bytes=64):
    """Determines the file version based on header content."""
    with open(file_path, 'rb') as f:
        header_bytes = f.read(num_bytes)
    return 2 if "v2" in header_bytes.decode('ascii', errors='replace') else 1

def process_file(file_path, force_offset=None):
    """
    Processes a TMD file while handling variations in height map data size.
    Allows forcing offsets by setting `force_offset=(x_offset, y_offset)`.
    """
    if not os.path.exists(file_path):
        return None, None

    version = check_version(file_path)
    with open(file_path, 'rb') as f:
        if version == 1:
            f.seek(28)  # Offset for v1 files
            print("⚠️ Detected v1 file format.")
        elif version == 2:
            f.seek(32)  # Offset for v2 files
            print("⚠️ Detected v2 file format.")
            # String 24 bytes long
            comment = f.read(24).decode('ascii').strip()
            print(f"Comment: {comment}")
        else:
            return None, None

        # Read dimensions and offsets
        width = struct.unpack('<I', f.read(4))[0]
        print(f"Width: {width}")
        height = struct.unpack('<I', f.read(4))[0]
        print(f"Height: {height}")
        x_length = struct.unpack('<f', f.read(4))[0]
        print(f"X Length: {x_length}")
        y_length = struct.unpack('<f', f.read(4))[0]
        print(f"Y Length: {y_length}")
        x_offset = struct.unpack('<f', f.read(4))[0]
        print(f"X Offset: {x_offset}")
        y_offset = struct.unpack('<f', f.read(4))[0]
        print(f"Y Offset: {y_offset}")

        # Apply forced offset if provided
        if force_offset:
            x_offset, y_offset = force_offset
            print(f"⚠️ Using forced offsets: x_offset={x_offset}, y_offset={y_offset}")

        # Read height map with flexibility in shape
        height_map_data = np.frombuffer(f.read(), dtype=np.float32)  # Read remaining data
        expected_size = width * height

        if height_map_data.size < expected_size:
            print(f"Warning: Incomplete height map. Expected {expected_size}, got {height_map_data.size}.")
            height_map = np.pad(height_map_data, (0, expected_size - height_map_data.size), mode='constant')
        elif height_map_data.size > expected_size:
            print(f"Warning: Extra data detected. Expected {expected_size}, got {height_map_data.size}. Trimming excess.")
            height_map = height_map_data[:expected_size]  # Trim excess data
        else:
            height_map = height_map_data

        height_map = height_map.reshape((height, width))

    metadata = {
        "version": version,
        "width": width,
        "height": height,
        "x_length": x_length,
        "y_length": y_length,
        "x_offset": x_offset,
        "y_offset": y_offset,
    }

    if version == 2:
        metadata["comment"] = comment

    return metadata, height_map

def plot_height_map(height_map, title="Height Map", filename="height_map.html"):
    """Creates and saves a 3D surface plot using Plotly."""
    fig = go.Figure(data=[go.Surface(z=height_map)])
    fig.update_layout(
        title=title,
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Height'),
        margin=dict(l=65, r=50, b=65, t=90)
    )
    fig.write_html(f"{filename}", include_plotlyjs="cdn")
    return f"{filename}"

# Set forced offsets (None means use extracted values)
FORCE_OFFSET = None  # Example: (0.22, 0.22) to override

# Process both files with optional forced offset
metadata_v1, height_map_v1 = process_file(file_paths["v1"], force_offset=FORCE_OFFSET)
metadata_v2, height_map_v2 = process_file(file_paths["v2"], force_offset=FORCE_OFFSET)

# Generate plots
plot_v1 = plot_height_map(height_map_v1, title="Height Map (v1)", filename="height_map_v1.html")
plot_v2 = plot_height_map(height_map_v2, title="Height Map (v2)", filename="height_map_v2.html")

# Display results
print("Metadata for v1:", metadata_v1)
print("Metadata for v2:", metadata_v2)

print("3D Plots saved:")
print(f" - [Height Map v1]({plot_v1})")
print(f" - [Height Map v2]({plot_v2})")


⚠️ Detected v1 file format.
Width: 300
Height: 300
X Length: 18.956600189208984
Y Length: 18.956600189208984
X Offset: 0.22351792454719543
Y Offset: 0.2231409251689911
⚠️ Detected v2 file format.
Comment: Created by TrueMap v6
 
Width: 300
Height: 300
X Length: 18.956600189208984
Y Length: 18.956600189208984
X Offset: 0.0
Y Offset: 0.0
Metadata for v1: {'version': 1, 'width': 300, 'height': 300, 'x_length': 18.956600189208984, 'y_length': 18.956600189208984, 'x_offset': 0.22351792454719543, 'y_offset': 0.2231409251689911}
Metadata for v2: {'version': 2, 'width': 300, 'height': 300, 'x_length': 18.956600189208984, 'y_length': 18.956600189208984, 'x_offset': 0.0, 'y_offset': 0.0, 'comment': 'Created by TrueMap v6\r\n\x00'}
3D Plots saved:
 - [Height Map v1](height_map_v1.html)
 - [Height Map v2](height_map_v2.html)
